This is a notebook version of the training script. I did not end up using this; instead I used main.py on a google cloud VM instance.

## Setup

In [1]:
# # Run this cell if running on Colab
# from pathlib import Path
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install transformers datasets

# import sys
# proj_dir = '/content/drive/MyDrive/Projects/sentiment_analysis'
# sys.path.append(proj_dir)
# proj_dir = Path(proj_dir)

In [2]:
# Run if running locally
from pathlib import Path
proj_dir = Path('.')

In [3]:
import torch
from datasets import load_from_disk, Dataset
from proj_utils import train, default_collator, bilingual_collator, RegularizedCLSModel, BilingualCLSModel
import gc

In [4]:
model_dir = proj_dir / 'models'
data_dir = proj_dir / 'data'
device = torch.device('cuda')

In [5]:
tl_model_name = "jcblaise/roberta-tagalog-base"
xlm_model_name = "xlm-roberta-base"
en_model_name = "bert-base-uncased"

## Model 1: roberta-tagalog-base (base text)

In [6]:
# tl_model_cls = AutoModelForSequenceClassification.from_pretrained(tl_model_name)
tl_model_cls = RegularizedCLSModel(tl_model_name)

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classi

In [12]:
tl_model_cls.model.classifier

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=2, bias=True)
)

In [8]:
list(tl_model_cls.model.modules())[0]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [7]:
dataset_1 = load_from_disk(data_dir / 'dataset_1')

In [10]:
# train(tl_model_cls, dataset_1, epochs=10, parameters=tl_model_cls.model.classifier.parameters(), 
#       model_dir=model_dir/'model_1/cls_only')

In [11]:
# for lr in [1e-4, 1e-5]:
#     tl_model_cls.load_state_dict(torch.load(model_dir/'model_1/cls_only/model_4.pt'))
#     train(tl_model_cls, dataset_1, epochs=10, model_dir=model_dir/f'model_1/all_params_{lr}', lr=lr)

In [12]:
del tl_model_cls
gc.collect()
torch.cuda.empty_cache()

## Model 2: XLM-RoBERTa-base

In [6]:
xlm_model_cls = RegularizedCLSModel(xlm_model_name)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [7]:
dataset_2 = load_from_disk(data_dir / 'dataset_2')

In [8]:
train(xlm_model_cls, dataset_2, epochs=5, batch_size=4, parameters=xlm_model_cls.model.classifier.parameters(), 
      model_dir=model_dir/'model_2/cls_only')


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

  0%|          | 0/13726 [00:00<?, ?batch/s]


  0%|                                                                                            | 0/5 [00:04<?, ?it/s]

torch.Size([4, 58])
0.6372441526249517


RuntimeError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 6.00 GiB total capacity; 4.82 GiB already allocated; 0 bytes free; 5.23 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# for lr in [1e-4, 1e-5]:
#     xlm_model_cls.load_state_dict(torch.load(model_dir/'model_2/cls_only/model_4.pt'))
#     train(xlm_model_cls, dataset_2, epochs=10, batch_size=4, model_dir=model_dir/f'model_2/all_params_{lr}', lr=lr)

In [ ]:
del xlm_model_cls
gc.collect()
torch.cuda.empty_cache()

## Model 3: bert-base-uncased (translated)

In [11]:
en_model_cls = RegularizedCLSModel(en_model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
dataset_3 = load_from_disk(data_dir / 'dataset_3')

In [13]:
train(en_model_cls, dataset_3, epochs=5, parameters=en_model_cls.model.classifier.parameters(), 
      model_dir=model_dir/'model_3/cls_only_1e-4', lr=1e-4)


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3432 [00:00<?, ?batch/s]

  0%|          | 0/382 [00:00<?, ?batch/s]


 20%|████████████████                                                                | 1/5 [18:10<1:12:40, 1090.14s/it]

  0%|          | 0/3432 [00:00<?, ?batch/s]

...

  0%|          | 0/382 [00:00<?, ?batch/s]


 80%|████████████████████████████████████████████████████████████████                | 4/5 [1:12:22<18:03, 1083.84s/it]

  0%|          | 0/3432 [00:00<?, ?batch/s]

  0%|          | 0/382 [00:00<?, ?batch/s]

100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [1:30:07<00:00, 1081.53s/it]


In [9]:
# for dropout in [0.1, 0.3, 0.5]:
#     for alpha in [0.1, 0.01, 0.001]:
#         en_model_cls = RegularizedCLSModel(en_model_name)
#         en_model_cls.load_state_dict(torch.load(model_dir/'model_3/cls_only/model_4.pt'))
#         train(en_model_cls, dataset_3, epochs=10, model_dir=model_dir/f'model_3/all_params_p_{dropout}_alpha_{alpha}', 
#               lr=1e-04, batch_size=8)

In [10]:
del en_model_cls
gc.collect()
torch.cuda.empty_cache()

## Model 4: Bilingual, Non-Translated

In [ ]:
non_translated_model = BilingualCLSModel(tl_model_name, en_model_name)

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/437M [00:00<?, ?B/s]

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'cla

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
dataset_4 = load_from_disk(data_dir / 'dataset_4')

In [ ]:
non_translated_model.tl_model.to('cuda')
non_translated_model.en_model.to('cuda')

In [ ]:
train(non_translated_model, dataset_4, epochs=5, parameters=non_translated_model.get_cls_parameters(), 
      model_dir=model_dir/'model_4/cls_only', collator=bilingual_collator)


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/403 [00:00<?, ?batch/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/5 [00:54<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
for dropout in [0.1, 0.3, 0.5]:
    for alpha in [0.1, 0.03, 0.01, 0.003]:
        non_translated_model = BilingualCLSModel(tl_model_name, en_model_name)
        non_translated_model.model.load_state_dict(torch.load(model_dir/'model_4/cls_only/model_4.pt'))
        train(non_translated_model, dataset_4, epochs=10, model_dir=model_dir/f'model_4/all_params_p_{dropout}_alpha_{alpha}', 
              lr=1e-04, collator=bilingual_collator)

In [ ]:
del non_translated_model
gc.collect()
torch.cuda.empty_cache()

## Model 5: Bilingual, Half-Translated

In [7]:
half_translated_model = BilingualCLSModel(tl_model_name, en_model_name)

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/437M [00:00<?, ?B/s]

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifi

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
dataset_5 = load_from_disk(data_dir / 'dataset_5')

In [9]:
# half_translated_model.tl_model.to('cuda')
# half_translated_model.en_model.to('cuda')

In [10]:
# train(half_translated_model, dataset_5, epochs=5, parameters=half_translated_model.get_cls_parameters(), 
#       model_dir=model_dir/'model_5/cls_only', collator=bilingual_collator)

In [ ]:
for dropout in [0.3, 0.5]:
    for alpha in [5, 1]:
        half_translated_model = BilingualCLSModel(tl_model_name, en_model_name, alpha=alpha, dropout=dropout)
        gc.collect()
        torch.cuda.empty_cache()
        half_translated_model.tl_model.to('cuda')
        half_translated_model.en_model.to('cuda')
        half_translated_model.load_state_dict(torch.load(model_dir/'model_5/cls_only/model_4.pt'))
        train(half_translated_model, dataset_5, epochs=10, model_dir=model_dir/f'model_5/all_params_p_{dropout}_alpha_{alpha}', 
              lr=1e-05, collator=bilingual_collator) 

Some weights of the model checkpoint at jcblaise/roberta-tagalog-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jcblaise/roberta-tagalog-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifi

  0%|          | 0/403 [00:00<?, ?batch/s]

  0%|          | 0/45 [00:00<?, ?batch/s]


 10%|█         | 1/10 [03:06<27:59, 186.64s/it]

  0%|          | 0/403 [00:00<?, ?batch/s]

...


 20%|██        | 2/10 [06:22<25:29, 191.14s/it]

  0%|          | 0/403 [00:00<?, ?batch/s]

  0%|          | 0/45 [00:00<?, ?batch/s]


 30%|███       | 3/10 [09:34<22:20, 191.52s/it]

  0%|          | 0/403 [00:00<?, ?batch/s]

In [ ]:
del half_translated_model
gc.collect()
torch.cuda.empty_cache()

## Model 6: Bilingual, Fully Translated

In [ ]:
full_translated_model = BilingualCLSModel(tl_model_name, en_model_name)

In [ ]:
dataset_6 = load_from_disk(data_dir / 'dataset_6')

In [ ]:
# full_translated_model.tl_model.to('cuda')
# full_translated_model.en_model.to('cuda')

In [ ]:
train(full_translated_model, dataset_6, epochs=5, parameters=full_translated_model.get_cls_parameters(), 
      model_dir=model_dir/'model_6/cls_only', collator=bilingual_collator)

In [ ]:
for dropout in [0.1, 0.3, 0.5]:
    for alpha in [0.1, 0.001]:
        full_translated_model = BilingualCLSModel(tl_model_name, en_model_name, alpha=alpha, dropout=dropout)
        gc.collect()
        full_translated_model.tl_model.to('cuda')
        full_translated_model.en_model.to('cuda')
        full_translated_model.load_state_dict(torch.load(model_dir/'model_6/cls_only/model_4.pt'))
        train(half_translated_model, dataset_6, epochs=10, model_dir=model_dir/f'model_6/all_params_p_{dropout}_alpha_{alpha}', 
              lr=1e-04, collator=bilingual_collator)

In [ ]:
del full_translated_model
gc.collect()
torch.cuda.empty_cache()